In [1]:
import os
# Set environment variables
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer
import torch

/media/.venv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/media/.venv/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
Q_GROUP_SIZE = 128 
ZERO_POINT = True 
W_BIT = 4 
VERSION = "GEMM" 
SAFETENSORS = True

quant_config = { "zero_point": ZERO_POINT, "q_group_size": Q_GROUP_SIZE, "w_bit": 4, "version": VERSION }

# Load model
model = AutoAWQForCausalLM.from_pretrained("google/gemma-2-9b", torch_dtype=torch.bfloat16, safetensors=SAFETENSORS)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-9b", trust_remote_code=True)

In [ ]:
# Quantize
model.quantize(tokenizer, quant_config=quant_config)

In [5]:
def pseudo_dequantize_tensor(w, scales, zeros = None):
        # get repeated count
        repeat_count = w.weight.data.shape[-1] // scales.shape[-1]
        scales = scales.repeat(1, repeat_count).reshape(w.weight.data.shape)

        # dequantize
        # if self.zero_point:
        zeros = zeros.repeat(1, repeat_count).reshape(w.weight.data.shape)
        w = (w.weight.data - zeros) * scales
        # else:
        # w = w.weight.data * scales

        return w

In [3]:
quant_path = "./gemma-2-9b-awq4"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(quant_path, trust_remote_code=True)

In [4]:
model_q = AutoAWQForCausalLM.from_quantized(quant_path, attn_implementation='eager')

Fusing layers...: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 42/42 [00:00<00:00, 192.81it/s]
